In [53]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import multivariate_normal
from scipy.stats import norm

np.random.seed(99)

## Problem 1

In [67]:
simulations = 10000
pd.set_option('display.float_format', lambda x: '%.4f' % x)

corr_matrix = np.eye(5)
# fill 1st column and first row with .05-.2 with a step of .05
for i in range(1, 5):
    corr_matrix[0, i] = corr_matrix[i, 0] = .05 * i

for i in range(2, 5):
    corr_matrix[1, i] = corr_matrix[i, 1] = .15 + .05 * i

for i in range(3, 5):
    corr_matrix[2, i] = corr_matrix[i, 2] = .25 + .05 * i

corr_matrix[3, 4] = corr_matrix[4, 3] = .5
comp_corr_matrix = np.eye(5)

mean = np.zeros(5)

cov_matrix = np.dot(np.dot(np.eye(5), corr_matrix), np.eye(5))
comp_cov_matrix = np.dot(np.dot(np.eye(5), comp_corr_matrix), np.eye(5))

prob_default = np.arange(0.1, 0.6, 0.1)
prob_default[::] = prob_default[::-1]

prob_default

array([0.5, 0.4, 0.3, 0.2, 0.1])

In [54]:
correlated_simulation = multivariate_normal.rvs(mean=mean, cov=cov_matrix, size=simulations)
simulation = multivariate_normal.rvs(mean=mean, cov=comp_cov_matrix, size=simulations)
correlated_simulation
# simulation

array([[ 1.84560957, -0.58894608, -0.98595586,  0.63440523,  0.36195295],
       [ 0.69760566,  0.17125337, -0.73713171, -0.96171351,  1.34141219],
       [ 0.66340545, -0.14634101, -0.1651917 ,  0.42907001,  0.041261  ],
       ...,
       [ 0.71223177, -0.23430259, -0.79931654,  0.13785999, -0.45890227],
       [ 1.20000254,  0.35894556,  0.93042362,  0.65998035, -0.24160086],
       [ 1.75637436,  0.66848911,  0.23093716,  0.0993917 ,  0.90815346]])

In [55]:
simulation_percentiles = norm.cdf(correlated_simulation)
comp_simulation_percentiles = norm.cdf(simulation)

In [57]:
default = np.zeros((simulations, 5))
comp_default = np.zeros((simulations, 5))

for i in range(simulation.shape[0]):
    for j in range(simulation.shape[1]):
        default[i, j] = 1 if simulation_percentiles[i, j] < prob_default[j] else 0
        comp_default[i, j] = 1 if comp_simulation_percentiles[i, j] < prob_default[j] else 0

default

array([[0., 1., 1., 0., 0.],
       [0., 0., 1., 1., 0.],
       [0., 0., 0., 0., 0.],
       ...,
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [63]:
default.std(axis=0).T

array([0.49997696, 0.48963079, 0.4561775 , 0.40208477, 0.30277977])

In [68]:
default_labels = ['Correlated Default', 'Uncorrelated Default']
firm_labels = ['Firm 1', 'Firm 2', 'Firm 3', 'Firm 4', 'Firm 5']
default.std(axis=0)
comp_default.std(axis=0)

default_df = pd.DataFrame([default.std(axis=0), comp_default.std(axis=0)], columns=firm_labels, index=default_labels)
default_df

,Firm 1,Firm 2,Firm 3,Firm 4,Firm 5
Correlated Default,0.5000,0.4896,0.4562,0.4021,0.3028
Uncorrelated Default,0.5000,0.4894,0.4586,0.4008,0.2987
